# Maritime Fleet Optimization under IMO 2023 Emission Trading Regulations using Reinforcement Learning

## Problem Description

Given:

1. A predetermined number of contracts for cargo transportation of a whole year,

2. a heterogeneous shipping fleet operating in tramp mode,  

find the optimal routing and scheduling policy in terms of USD.

When solving for optimality take into account the penalties imposed for excessive C02 emissions.

### Terminology

* **Heterogeneous fleet** is a fleet containing ships of different characteristics e.g. type, capacity, engine power etc.

* **Tramp fleets** are fleets that take on contracts to transport specified volumes of cargo between two ports within a specified period of time.

* **Ballast trips** are trips made to pick up ports by an empty ship to pick up a cargo. 

### Problem Characteristics and Assumptions

* There exists a set of contracts that the fleet must serve in a certain planning horizon. This set is completely known at the beginning of the horizon.

* Each contract is characterized by:

    * a single trip or cargo

    * an origin-destination pair of ports

    * time windows for both picking and delivering the cargo. 

* A ship can serve at most one contract at a time.

* Fleet is non-homogeneous. Ship capacity, navigation speeds, fuel consumption, cost parameters etc are all ship dependent. 

* Due to cargo-ship or ship-port incompatibilities not all ships can serve all contracts.

* Two cargoes may be incompatible with each other, meaning that corresponding trips cannot be done consecutively by the same ship, unless a time delay (e.g. for cleaning) or a third trip is placed between them.

* Navigation speed is a controllable variable. Since fuel consumption depends on navigation speed, controlling the speed impacts not only travel time, but also travel costs. Since navigation speed affects fuel consumption, it also determines CO2 emissions and ultimately the Carbon Intensity Indicator. In the paper *Ronen D (1993) Ship scheduling: the last decade* **fuel consumption is approximately a cubic function of speed**.

* Income from each contract is fixed, so net profit is maximized by minimizing total relevant costs. Relevant costs are mainly operating costs, but may also include other kinds of costs as long as they can be associated with each individual trip.

* It is important to notice that given a sequence of cargoes to be served by a single ship, a ballast or empty trip must take place from the delivery port of each cargo to the origin of the next cargo in the sequence, unless these two ports coincide. **These ballast trips represent a singificant portion of total avoidable cost.**


The problem consists of : 

1. Deciding the set of cargoes to be served by each ship;

2. Determining the time of arrival at a port;

3. Determining the time of departure from a port;

4. Determining the waiting times at each port;

5. Determining the navigation speed at each trip;

The **objective** is to serve all cargoes while minimizing the total relevant operating costs.

Operating costs include:

* Fuel Consumption

* Port and Custom Expenses

* Tolls paid at canals (Suez and Panama)

Operating costs depend on:

* Traveled distance

* Navigation speed

* Maritime routes

## Modeling the problem

The problem's modeling is based on the work done in the paper: [A Time Based Discretization Approach for Ship Routing and Scheduling with Variable Speed](https://link.springer.com/article/10.1007/s11067-010-9132-9)

The main strategy to build the model is a discrete segmentation of the time windows for picking and delivering the cargoes.

If range $A$ represents the time windows for picking a cargo, the authors assume that arrival to the port of origin (beginning of service) may happen only at a discrete set of times

$A = \left \{ a_1 , a_2,\dots ,a_m \right \}$ with $a_i \in A$ for all $i=1,2,\dots,m$ 

Delivery time windows are treated similarly.

The authors use:

* $n = 1,...,N$ to represent a predetermined number of $N$ cargo contracts for a year.
* $k = 1,..,B$ to represent a fleet of $B$ heterogeneous vessels. 
* $v = 1,...,V$ to represent a number of $v$ discretized time step nodes.
* $D_n$ to represent the possible departure times for each trip associated with cargo $n$ 
* $A$ to represent the set of possible arcs in the network
* $c_{ijk}$ to represent the cost of the arc $(i,j,k)$
* $x_{ijk} = \left\{\begin{matrix}
1 &\quad\text{if arc (i,j,k) is selected as part of the solution } \\ 
0 &\quad\text{otherwise}
\end{matrix}\right.
$ 




Under these assumptions, the problem can be modeled by a graph similar to the one in the figure below.

![](ship1.png)


Each small full circle node represents a possible time node to begin the service of a specific cargo. 

For each node $i$ let $n(i)$ be the cargo associated with the node $i$. 

Given nodes $i$ and $j$, there is an $arc (i,j,k)$ from $i$ to $j$ if ship $k$ is able to serve trips $n(i)$ and $n(j)$ consecutively.

For this to be true the following must hold:

* Cargo $n(i)$ is compatible with cargo $n(j)$
* Both cargoes are compatible with ship $k$
* If ship $k$ begins service of cargo $n(i)$ at the time instance represented by node $i$, it can deliver the cargo within the corresponding time windows, make the ballast trip from the destination port of cargo n(i) to the origin port of cargo $n(j)$ and be available to begin service of cargo $n(j)$ at the time instance associated with node $j$.  


With each arc $(i,j,k)$ we associate a cost parameter $c_{i,j,k}$ , which represents the minimum total cost incurred if ship $k$ serves cargo $n(i)$ immediately followed by cargo $n(j)$. 

This cost must be computed a priori outside the model and includes :

1. Reaching the origin port of n(i) via a ballast trip
2. Calculating possible waiting times there
3. Making the delivery trip to the destination port of n(i)
4. Making a ballast trip to the origin port of $n(j)$

**Notice that to compute each parameter $c_{i,j,k}$ the optimal combination of navigation speeds on the trips of $n(i)$ and on the ballast trip for $n(j)$ must be determined, which in fact may require the solution of another optimization problem.**


To complete the graph, the authors add a fictitious node $0$  that represents the starting point of all ships.

For each ship $k$ and node $i$, two arcs $(0,i,k)$ and $(i,0,k)$ exist from the fictitious node $0$  to node $i$ and vice versa, if the cargo $n(i)$ is compatible with ship $k$.

* Cost $c_{0,i,k}$ is calculated based on the real initial position of ship $k$.
* Cost $c_{i,0,k}$ represents the minimum total cost incurred if ship $k$ actually serves cargo $n(i)$

Assuming that ship $k$ will stay at the destination port of its last assigned cargo, no ballast trip is considered in $c_{i,0,k}$ . 


Figure 2 shows a solution for an example with 5 cargoes to be served and 2 ships to be used.

![](ship2.png)



Ship 1 serves cargo 2 and cargo 4. The trip for cargo 2 begins at the time presented by node 6 and the trip for cargo 4 begins at the time represented by node 16. 

Ship 2 serves cargo 1 , cargo 3 and cargo 5. The trip for cargo 1 begins at the time represented by node 4, then the trip for cargo 3 starts at node 10 and finally the trip for cargo 5 starts at node 17. 

### Mathematical Formulation and Interpretation

The mathematical formulation of the problem described in the paper is as follows:


$$ Min  \sum_{\left ( i,j,k \right ) \in A}^{} c_{ijk} \, \cdot x_{ijk} $$

s.t. : 

$$ \sum_{ i \in V / \left ( 0,i,k \right ) \in A}^{} x_{0ik} \leq 1 \; \; k = 1,\cdots , B $$


$$ \sum_{  \left ( i,j,k \right  ) \in A / j \in D_n }^{} x_{ijk} = 1 \; \; n = 1,\cdots , N $$

$$  \sum_{ i \in V / \left ( i,j,k \right ) \in A}^{} x_{ijk} = \sum_{ l \in V / \left ( j,l,k \right ) \in A}^{} x_{jlk}  \; \; j \in V , \; \; k = 1,\cdots , B  $$



---

In this section we give an interpretation of each equation



**Objective Function**  

$$ Min  \sum_{\left ( i,j,k \right ) \in A}^{} c_{ijk} \, \cdot x_{ijk} $$

**Interpretation** : Minimize the total cost of the arcs selected as solutions


**Constraint**

$$ \sum_{ i \in V / \left ( 0,i,k \right ) \in A}^{} x_{0ik} \leq 1 \; \; k = 1,\cdots , B $$

**Interpretation** : Each ship is employed at most in one route, consisting of a sequence of cargoes to be served.

![](ship3.png)

**Constraint**

$$ \sum_{  \left ( i,j,k \right  ) \in A / j \in D_n }^{} x_{ijk} = 1 \; \; n = 1,\cdots , N $$

**Interpretation** : Each cargo must be served exactly once, by exactly one ship, which begins service at exactly one of the nodes in the discritized time window $D_n$  

**Constraint**

$$ \sum_{ i \in V / \left ( i,j,k \right ) \in A}^{} x_{ijk} = \sum_{ l \in V / \left ( j,l,k \right ) \in A}^{} x_{jlk}  \; \; j \in V , \; \; k = 1,\cdots , B  $$

**Interpretation** : For nodes different than the central fictitious node 0, if an entering arc is selected, a leaving arc must also be selected and in such case, both arcs must be associated with the same ship.

## Important Remarks

The authors say that **the cost parameters for each potential arc have to be determined outside the model, beforehand, probably by using another optimization model.**

Then they should be used as inputs to this model. 

For example, if navigation speed is considered, the cost parameter of an arc must reflect the minimum achievable cost over all feasible speed combinations on the contracted trip and the ballast trip associated with the arc. This minimization, however, should be performed by an external model whose output (the arc cost) enters as a constant for this model.

In general, more than one arc can represent the same pair of trips. These arcs differ on the starting times (nodes) for each trip, which imply that minimum cost may be achieved at different speeds for different arcs. So when selecting the arcs that constitute a route, the model is implicitly optimizing the navigation speed for each leg of the route. 



**Do you have any suggestion regarding the way we can determine the optimal navigation speeds for each route?**

Reminder:

![](pic4.png)


Fuel Consumption is considered a cubed function of Navigation Speed in the relevant bibliography.

Moreover since Fuel Consumption affects the Carbon Intensity Indicator it is vital for us to find a proper way to set the Navigation Speed for the different routes as input for our model. 